# Reading the rest of HYDRUS files [Stage 2]
![The project files](../Assets/Project_files_2.png)

## Importing and loading dataset

In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from NesrHydrusAnalyst import *

In [2]:
src = '../Datasets/H3D2_SandDitch0011'

## Unreadable files
Encrypted files that are converted to text elswhere
1. `h.out`
2. `Q.out`
3. `v.out`
3. `th.out`
4. `MESHTRIA.000`
4. `DOMAIN.IN`

## Reading previous implemented files
### Reading the _mesh files_
1. `H.TXT`
2. `V.TXT`
3. `TH.TXT`
4. `MESHTRIA.TXT`
### Spread-info files

1. `Cum_Q.out`
1. `h_Mean.out`
1. `v_Mean.out`
5. `Run_Inf.out`
### the _information files_
1. `A_Level.out`
2. `ATMOSPH.IN`
3. `Balance.out`
4. `DIMENSIO.IN`
5. `Run_Inf.out` (Repeated)
6. `SELECTOR.IN`

## Files to be implemented here


### Flux-info files

1. `Boundary.out`
1. `BOUNDARY.IN`
1. `Check.out`

In [ ]:
folder =src

# def read_boundary_data(folder='Current', save_to_csv=True,
#                      read_velocities=False):
#     '''
#     A function to read both Theat and H files from HYDRUS outputs, 
#         then to:
#             1- return one dataframe contains both data in a decent format.
#             2- save this output to a CSV file (optional, True by default)
#     Input:
#         The name of the main folder (leave balank for the current folder)
#         The option to save_to csv, default =True (Boolean)
#     '''

# Specify the source folder
if folder == 'Current':
    read_dir = os.getcwd()
else:
    read_dir = folder



# Finding number of nodes in the file
mesh_file = os.path.join(read_dir, 'BOUNDARY.IN')
# num_cells = np.array(linecache.getline(mesh_file, 6).split(), int)#[0]
linecache.getline(mesh_file, 6).split()
# # Define dataframe titles
# titles = ['n', 'x', 'y', 'z']
# # Define a list of coordinates
# full_data = [[0, 0, 0, 0]]
# # Set a loop to geather all coordinates from MESHTRIA.TXT file
# for i in range(8, num_cells + 8):
#     full_data.append(
#         np.array(linecache.getline(mesh_file, i).split(), float))
# # Convert the list to numpy array then to a dataframe
# coordinates_df = pd.DataFrame(np.array(full_data), columns=titles)

In [29]:
folder = src

# def read_boundary_data(folder='Current', save_to_csv=True,
#                      read_velocities=False):
#     '''
#     A function to read both Theat and H files from HYDRUS outputs,
#         then to:
#             1- return one dataframe contains both data in a decent format.
#             2- save this output to a CSV file (optional, True by default)
#     Input:
#         The name of the main folder (leave balank for the current folder)
#         The option to save_to csv, default =True (Boolean)
#     '''

# Specify the source folder
if folder == 'Current':
    read_dir = os.getcwd()
else:
    read_dir = folder

# Finding number of nodes in the file
mesh_file = os.path.join(read_dir, 'BOUNDARY.IN')
num_cells = int(linecache.getline(mesh_file, 4).split()[0])
# Define dataframe titles
titles = ['n', 'surface_area']


def get_num_lines(num_cells, nums_per_line=10):
    num_lines = int(num_cells / nums_per_line)
    if num_cells % nums_per_line > 0: num_lines += 1
    return num_lines


def read_snakey_list(file_name, start_line, end_line, data_type=float):
    '''
    Reading a list of numbers thar are stored in a text file, where
    there are `numbers_count` numbers, stored as `nums_per_line` numbers 
    per line, the starting position of first occurence of numbers is at
    `start_line` line number
    returns a numpy array of the data with the `data_type` given
    '''
    points = []
    for i in range(start_line, end_line):
        points.extend(linecache.getline(file_name, i).split())
    return np.array(points, data_type)


num_lines = get_num_lines(num_cells, nums_per_line)

first_line = 10
end_line = first_line + num_lines
points = read_snakey_list(mesh_file, first_line, end_line, data_type=int)

first_line = end_line + 1
end_line = first_line + num_lines
areas = read_snakey_list(mesh_file, first_line, end_line)
data = np.array([points, areas]).T
data_in = pd.DataFrame(
    data, columns=titles).astype({
        "n": int,
        "surface_area": float
    })
data_in
# pd.DataFrame([points, areas], index=titles).T
#     num_lines , start_line, num_lines + start_line, points
#     points.append(np.array(linecache.getline(mesh_file, i).split(), int))
# # Convert the list to numpy array then to a dataframe
# coordinates_df = pd.DataFrame(np.array(full_data), columns=titles)

#     # Print head and tail of the dataframe to ensure correctness
#     # pd.concat([coordinates_df.head(),coordinates_df.tail()])

,n,surface_area
0,1,0.020491
1,8,0.040982
2,9,0.762819
3,11,1.275030
4,12,0.512820
...,...,...
513,3152,1.538460
514,3153,1.538460
515,3154,1.538460
516,3155,1.538460


In [4]:
#     # -----------------------------#
#     # To get data from all files   #
#     # -----------------------------#
#     def get_data_from_file(filename='TH.TXT', main_caption='Theta'):
#         '''
#         Function to combine all values of a property to a single dataframe 
#         inputs:
#         filename, the name of the file
#         caption, the leading caption of the columns (we will add the portion '_T= xxx')
#         where xxx is the timestep
#         '''
#         # compute number of lines for each timestep
#         num_lines = int(math.ceil(num_cells / 10.))
#         time_steps_remaining = True  # Flag to see if the loop should continue or not.
#         times_df = pd.DataFrame([])  # Empty dataframe
#         time_loc_start = 2  # The starting cell of the timestep
#         # Check if it is a velocity file
#         processing_velocity = (filename[-5:] == 'V.TXT')
#         while time_steps_remaining:
#             line_t = linecache.getline(filename, time_loc_start).split()
#             # Check if it is the start of the timestep, otherwise exit
#             if line_t[0] == 'Time':
#                 t = int(line_t[2])
#                 if processing_velocity:
#                     velocity_component = {
#                         'first': '1',
#                         'second': '2',
#                         'third': '3'
#                     }[line_t[5].strip()]
#                     caption = main_caption + velocity_component
#                 else:
#                     caption = main_caption
#                 # Finding the last line of the timestep
#                 tim_loc_end = num_lines + time_loc_start + 2
#                 # The starting time is always 0 because steps starts in 1 in HYDRUS
#                 time_data = [0]
#                 # Create the timestep as one long list
#                 for i in range(time_loc_start + 2, tim_loc_end):
#                     time_data.extend(linecache.getline(filename, i).split())
#                 # Convert the list to DataFrame
#                 dft = pd.DataFrame(
#                     np.array(time_data, float),
#                     columns=['{}_T{}'.format(caption, t)])
#                 if len(times_df) == 0:  # If it is the first timestep
#                     times_df = dft.copy()
#                 else:  # Otherwise (for all other timesteps)
#                     times_df = pd.concat([times_df, dft], axis=1)
#                 # Change the start to the probable next timestem (if exist)
#                 time_loc_start = tim_loc_end + 1
#                 time_steps_remaining = True if len(
#                     linecache.getline(filename, time_loc_start)) > 0 else False
#                 # End IF
#         return times_df

#     # Set the basic dataframe to the coordinates dataframe, to append to it.
#     full_df = coordinates_df
#     # Looping through the basic output files then to concatenate them all
#     properties = [('TH.TXT', 'Th'), ('H.TXT', 'H'), ('V.TXT', 'V')]
#     if not read_velocities:
#         properties = properties[:-1]
#     for prop in properties:
#         file_path = os.path.join(read_dir, prop[0])
#         # Check if the file exists
#         if os.path.isfile(file_path):
#             prop_df = get_data_from_file(file_path, prop[1])
#             full_df = pd.concat([full_df, prop_df], axis=1)
#         else:
#             print(
#                 'Warning, the file {} does not exist in the given path'.format(
#                     prop[0]))

#     # Convert the num column to integer
#     full_df[['n']] = full_df[['n']].astype(np.int64)
#     # dropping the first row (the zeros row) as it is not necessary
#     full_df.drop(0, inplace=True)
#     # Saving the resultant dataframe to disk.
#     if save_to_csv:
#         full_df.to_csv(os.path.join(read_dir, 'nesr_data2.csv'))
#     return full_df

['f', 'f', 'f', 'f', 'f', 'f', 'f', 'f']